In [14]:
import pandas as pd
import numpy as np

In [15]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [16]:
df = pd.read_csv('data/Train_rev1.csv')

In [ ]:
df.shape

In [18]:
df.head(3)

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Id                  244768 non-null  int64 
 1   Title               244767 non-null  object
 2   FullDescription     244768 non-null  object
 3   LocationRaw         244768 non-null  object
 4   LocationNormalized  244768 non-null  object
 5   ContractType        65442 non-null   object
 6   ContractTime        180863 non-null  object
 7   Company             212338 non-null  object
 8   Category            244768 non-null  object
 9   SalaryRaw           244768 non-null  object
 10  SalaryNormalized    244768 non-null  int64 
 11  SourceName          244767 non-null  object
dtypes: int64(2), object(10)
memory usage: 22.4+ MB


In [20]:
df.describe(include='all')

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
count,2.447680e+05,244767,244768,244768,244768,65442,180863,212338,244768,244768,244768.000000,244767
unique,NaN,135435,242138,20986,2732,2,2,20812,29,97286,NaN,167
top,NaN,Business Development Manager,What is expected of you as a Registered Nurse ...,London,UK,full_time,permanent,UKStaffsearch,IT Jobs,"50,000-74,999 yearly",NaN,totaljobs.com
freq,NaN,921,18,15605,41093,57538,151521,4997,38483,1923,NaN,48149
mean,6.970142e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34122.577576,NaN
std,3.129813e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17640.543124,NaN
min,1.261263e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5000.000000,NaN
25%,6.869550e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21500.000000,NaN
50%,6.993700e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.000000,NaN
75%,7.162606e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42500.000000,NaN


In [21]:
df.shape

(244768, 12)

# 1. Drop columns

In [22]:
df.drop(columns=['Id', 'SalaryRaw', 'LocationRaw'], inplace=True)

# 2. Fill missing values

In [23]:
print('Missing values:')
df.isna().sum()/df.shape[0]*100

Missing values:


Title                  0.000409
FullDescription        0.000000
LocationNormalized     0.000000
ContractType          73.263662
ContractTime          26.108397
Company               13.249281
Category               0.000000
SalaryNormalized       0.000000
SourceName             0.000409
dtype: float64

In [24]:
def fill_missing(df):
    for col in df.columns:
        if df[col].dtype == 'O':  # object/string
            df = df.fillna(df[col].mode()[0])
        else:  # numbers
            df = df.fillna(df[col].mean())
    return df

In [25]:
df = fill_missing(df)

In [26]:
df.isna().sum()/df.shape[0]*100

Title                 0.0
FullDescription       0.0
LocationNormalized    0.0
ContractType          0.0
ContractTime          0.0
Company               0.0
Category              0.0
SalaryNormalized      0.0
SourceName            0.0
dtype: float64

# 3. Duplicates

In [27]:
df.duplicated().sum()

1

# 4. Geostandarization

## 4.1. Get population data from geonames dataset

In [28]:
# selecting data only for GB - turn on once (long)
# cols = [
#     'geonameid','name','asciiname','alternatenames','lat','lon',
#     'feature_class','feature_code','country_code','cc2','admin1',
#     'admin2','admin3','admin4','population','elevation','dem','tz','moddate'
# ]

# geonames = pd.read_csv(
#     "allCountries.txt",
#     sep="\t",
#     names=cols,
#     usecols=['asciiname', 'alternatenames', 'country_code', 'feature_code', 'feature_class', 'admin1', 'admin2', 'admin3', 'lon', 'lat', 'population'],
#     dtype=str,
#     header=None
# )

# geonames_gb = geonames[geonames['country_code'] == 'GB'].copy().reset_index(drop=True)
# geonames_gb = geonames_gb[geonames_gb['feature_class'].isin(['P', 'A'])].reset_index()
# geonames_gb.loc[geonames_gb['feature_code'] == 'PCLI', 'asciiname'] = 'UK'
# geonames_gb.to_csv('geonames_gb.csv')

In [29]:
geonames_gb = pd.read_csv('geo_datasets/geonames_gb.csv')
geonames_gb.rename(columns={'asciiname': 'name'}, inplace=True)

## 4.2. Get population for all locations where it is directly possible

In [30]:
# get population for locations
pop_dict = geonames_gb['population'].copy()
pop_dict = geonames_gb.set_index(geonames_gb['name'].str.lower().str.strip())['population'].to_dict()

df['LocationPopulation'] = df['LocationNormalized'].str.lower().str.strip().map(lambda x: pop_dict.get(x))

In [31]:
def print_missing_info():
    print(f"Missing data in population of location: {round(df[df['LocationPopulation'].isna()]['LocationNormalized'].count() / len(df) * 100, 2)}%, {df[df['LocationPopulation'].isna()]['LocationNormalized'].count()} cases")
    print()
    print(df[df['LocationPopulation'].isna()]['LocationNormalized'].value_counts()[:5])

In [32]:
print_missing_info()

Missing data in population of location: 12.44%, 30460 cases

LocationNormalized
South East London    11713
Central London        2607
West Midlands         2540
Berkshire             1502
West Yorkshire        1072
Name: count, dtype: int64


## 4.3. Remove directions and assign population to other fitting names

In [33]:
directions = ['North', 'South', 'East', 'West', 'Central']
df['LocationNormalized'] = df['LocationNormalized'].replace(
    directions, '', regex=True
).str.strip()

missing_mask = df['LocationPopulation'].isna()
missing_locations = df.loc[missing_mask, 'LocationNormalized'].str.lower().str.strip()

pop_dict_missing = {loc: pop_dict.get(loc, np.nan) for loc in missing_locations}

df.loc[missing_mask, 'LocationPopulation'] = missing_locations.map(pop_dict_missing)

In [34]:
print_missing_info()

Missing data in population of location: 4.58%, 11216 cases

LocationNormalized
Midlands                    3456
Berkshire                   1502
Cheshire                     871
Yorkshire and Humberside     683
Bedfordshire                 544
Name: count, dtype: int64


## 4.4. Find population for Midlands in NUT regions

In [35]:
# remove locations out of GB
uk_lat_mask = (geonames_gb['lat'] >= 49) & (geonames_gb['lat'] <= 61)
uk_lon_mask = (geonames_gb['lon'] >= -10) & (geonames_gb['lon'] <= 2)
geonames_gb = geonames_gb[(geonames_gb['country_code'] == 'GB') & (uk_lat_mask) & (uk_lon_mask)]

In [39]:
nuts = pd.read_excel("geo_datasets/NUTS.xlsx")
nuts['NUTS118NM'] = nuts['NUTS118NM'].str.replace('(England)', '', regex=False).str.strip()
nuts = nuts.rename(columns={'NUTS118NM': 'name', 'LONG': 'lon', 'LAT': 'lat'})

In [40]:
# find the closest point in geonames in nuts
from scipy.spatial import cKDTree
tree = cKDTree(geonames_gb[['lat', 'lon']].values)
nuts_coords = nuts[['lat', 'lon']].values

distances, indices = tree.query(nuts_coords, k=1)  # k=1 -> 1 neighbour

nuts['population'] = geonames_gb.iloc[indices]['population'].values
nuts_population = dict(zip(nuts['name'], nuts['population']))

In [41]:
from typing import Counter

# combine West and East Midlands
nuts_population = {**{k: v for k, v in nuts_population.items() if 'Midlands' not in k},
                 **{'Midlands': sum(v for k, v in nuts_population.items() if 'Midlands' in k)}}

In [42]:
nuts_population

{'North East': 1126,
 'North West': 0,
 'Yorkshire and The Humber': 0,
 'East of England': 686,
 'London': 10750,
 'South East': 0,
 'South West': 0,
 'Wales': 0,
 'Scotland': 8830,
 'Northern Ireland': 0,
 'Midlands': 50878}

In [43]:
# impute nuts locations
population_from_dict = df['LocationNormalized'].map(nuts_population)

mask = ((df['LocationPopulation'].isnull()) | (df['LocationPopulation'] == 0)) & population_from_dict.notnull()

df.loc[mask, 'LocationPopulation'] = population_from_dict[mask]

In [44]:
print_missing_info()

Missing data in population of location: 3.17%, 7760 cases

LocationNormalized
Berkshire                   1502
Cheshire                     871
Yorkshire and Humberside     683
Bedfordshire                 544
Edinburgh Technopole         408
Name: count, dtype: int64


## 4.5. Cast rest of cases as 'UK'

In [45]:
mask = df['LocationPopulation'].isna() | (df['LocationPopulation'] == 0)
uk_pop = df.loc[df['LocationNormalized'].str.lower().eq('uk'), 'LocationPopulation'].dropna().iloc[0] if any(df['LocationNormalized'].str.lower().eq('uk')) else np.nan
df.loc[mask, ['LocationNormalized', 'LocationPopulation']] = ['UK', uk_pop]

In [46]:
print_missing_info()

Missing data in population of location: 0.0%, 0 cases

Series([], Name: count, dtype: int64)


In [47]:
df['LocationPopulation'].value_counts().head()

LocationPopulation
66488991.0    109313
8961989.0      45511
541263.0        3516
50878.0         3456
1157603.0       3061
Name: count, dtype: int64

In [48]:
df.drop(columns=['LocationNormalized'], inplace=True)

# 5. Split data

In [49]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, random_state=42)

# 6. One hot encoding

In [50]:
# select most common source in category group
category_to_source = train.groupby('Category')['SourceName'].agg(lambda x: x.mode()[0]).to_dict()
train['SourceName'] = train['Category'].map(category_to_source)
test['SourceName'] = test['Category'].map(category_to_source)

In [51]:
train = pd.get_dummies(train, columns = ['ContractType', 'ContractTime', 'Category', 'SourceName'], drop_first=True, dtype=int)
test = pd.get_dummies(test, columns = ['ContractType', 'ContractTime', 'Category', 'SourceName'], drop_first=True, dtype=int)

# 7. Target Encoding - mean salary of company instead of company name

In [52]:
# combining companies by two first words
train['CompanyPrefix'] = train['Company'].apply(lambda x: ' '.join(str(x).split()[:2]))
test['CompanyPrefix'] = test['Company'].apply(lambda x: ' '.join(str(x).split()[:2]))

In [53]:
# mean salary by company
mean_company = train.groupby('CompanyPrefix')['SalaryNormalized'].mean()
train['CompanyEncoded'] = train['CompanyPrefix'].map(mean_company)
test['CompanyEncoded'] = test['CompanyPrefix'].map(mean_company)

# filling not existing companies in test with global mean
global_mean = train['SalaryNormalized'].mean()
test['CompanyEncoded'] = test['CompanyEncoded'].fillna(global_mean)

train.drop(columns=['Company', 'CompanyPrefix'], inplace=True)
test.drop(columns=['Company', 'CompanyPrefix'], inplace=True)

In [54]:
mean_company.head(100)

CompanyPrefix
.Michael Page               77500.000000
1 1                         21040.000000
10 TRINITY                  45000.000000
100 percent                 40500.000000
100% IT                     30250.000000
121 International           47500.000000
121 Social                  20880.000000
16 West                     34625.000000
1Four1 Recruitment.co.uk    14976.000000
1st Central                 39083.333333
1st Choice                  22227.826087
1st Contact                 40000.000000
1st Executive               48507.291667
1st For                     26286.000000
1st Place                   24442.400000
1st Step                    36596.923077
1st Stop                    28208.250000
20 20                       35083.333333
2012 Security               13440.000000
2013 01                     20000.000000
2020 Technology             39500.000000
22 Recruitment              22500.000000
235 Consulting              50000.000000
24 7                        57654.666667
24

# 8. Standarization

In [55]:
from sklearn.preprocessing import StandardScaler
numeric_cols = ['SalaryNormalized', 'CompanyEncoded', 'LocationPopulation']
scaler = StandardScaler()
train[numeric_cols] = scaler.fit_transform(train[numeric_cols])
test[numeric_cols] = scaler.transform(test[numeric_cols])

# 9. Embeddings

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

tfidf_desc = TfidfVectorizer(max_features=5000, stop_words='english')
tfidf_title = TfidfVectorizer(max_features=1000, stop_words='english')

X_train_desc = tfidf_desc.fit_transform(train["FullDescription"])
X_test_desc  = tfidf_desc.transform(test["FullDescription"])

X_train_title = tfidf_title.fit_transform(train["Title"])
X_test_title  = tfidf_title.transform(test["Title"])

# join
X_train_text = hstack([X_train_desc, X_train_title])
X_test_text  = hstack([X_test_desc, X_test_title])

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(df['FullDescription'])
print(len(vectorizer.vocabulary_))

198677


In [58]:
vectorizer = CountVectorizer()
vectorizer.fit(df['Title'])
print(len(vectorizer.vocabulary_))

18530


In [ ]:
from sklearn.decomposition import TruncatedSVD

# dimenshion reduction
svd = TruncatedSVD(n_components=50, random_state=42)

X_train_text = svd.fit_transform(X_train_text)
X_test_text  = svd.transform(X_test_text)

In [ ]:
train.drop(columns=['Title', 'FullDescription'], inplace=True)
test.drop(columns=['Title', 'FullDescription'], inplace=True)

# 10. Data saving

In [ ]:
train.to_csv('data/train_preprocessed.csv', index=False)
test.to_csv('data/test_preprocessed.csv', index=False)

In [ ]:
train.head()

In [ ]:
import joblib
np.save("data/X_train_text.npy", X_train_text)
np.save("data/X_test_text.npy", X_test_text)